## 5.5 RNNLMの学習と評価

RNNLMを実装し、実際に学習させてみる。

### 5.5.1 RNNLMの実装

RNNLMで使用するネットワークを`SimpleRnnlm`というクラス名で実装する。
レイヤ構成は図5-30参照。



In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *


class SimpleRnnlm:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        # 重みの初期化
        embed_W = (rn(V, D) / 100).astype('f')
        rnn_Wx = (rn(D, H) / np.sqrt(D)).astype('f')  # np.sqrt(D)で割っているところがポイント
        rnn_Wh = (rn(H, H) / np.sqrt(H)).astype('f')
        rnn_b = np.zeros(H).astype('f')
        affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        # レイヤの生成
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful=True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]

        # すべての重みと勾配をリストにまとめる
        self.params, self.grads = [], []
        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, ts):
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

    def reset_state(self):
        self.rnn_layer.reset_state()

- Truncated BPTTで学習を想定して、stateful=True
- Xavierの初期値を利用
  - 前層のノードの個数を$n$とした場合に、$1/\sqrt{n}$の標準偏差を持つ分布を使う
    - 但しこれは簡略版の実装であり、オリジナルでは次層ノード数も考慮
  - 本書ではこれ以降も重みの初期値として「Xaiverの初期値」を用いる
- ディープラーニングでは重みの初期値が重要（ｂｙさそりさん）
- 順伝搬と逆伝搬は自明

### 5.5.2 言語モデルの評価

言語モデルの「評価方法」としてパープレキシティ（preplexity）を用いる。
パープレキシティは簡単にいうと「正解確率の逆数」あるいは「分岐数」。
例えば正解となる単語の予測確率が0.8ならばパープレキシティは1.25（低い方が良いモデル）。

入力データが複数の場合でも定義できて、
交差エントロピー（ニューラルネットワークの損失）$L = 1/N \sum_n \sum_k t_{nk} \log y_{nk}$に対して、
$e^L$をパープレキシティとする（$t_n$はone-hotの正解ベクトルなので$N=1$のときは正解確率の逆数になっている）。

情報理論の分野では「平均分岐数」とも呼ばれる・

### 5.5.3 RNNLMの学習コード

PTBデータセットを利用してRNNLMの学習を行う。
ただし、PTBデータセットの先頭の1000単語だけを利用することにする。
ここで実装したRNNLMでは全ての訓練データを対象にすると全然良い結果を出せない（だめじゃん）。

この改良は次章で行う（土田君ががんばる、私はがんばらない）。

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
import numpy as np
from common.optimizer import SGD
from dataset import ptb

# ハイパーパラメータの設定
batch_size = 10
wordvec_size = 100
hidden_size = 100
time_size = 5  # Truncated BPTTの展開する時間サイズ
lr = 0.1
max_epoch = 100

# 学習データの読み込み（データセットを小さくする）
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size]
vocab_size = int(max(corpus) + 1)

xs = corpus[:-1]  # 入力
ts = corpus[1:]  # 出力（教師ラベル）
data_size = len(xs)
print('corpus size: %d, vocabulary size: %d' % (corpus_size, vocab_size))

# 学習時に使用する変数
max_iters = data_size // (batch_size * time_size)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# モデルの生成
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

# ミニバッチの各サンプルの読み込み開始位置を計算
jump = (corpus_size - 1) // batch_size # この辺がTruncated BPTTっぽいところ
offsets = [i * jump for i in range(batch_size)]

for epoch in range(max_epoch):
    for iter in range(max_iters):
        # ミニバッチの取得
        batch_x = np.empty((batch_size, time_size), dtype='i')
        batch_t = np.empty((batch_size, time_size), dtype='i')
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                batch_x[i, t] = xs[(offset + time_idx) % data_size]
                batch_t[i, t] = ts[(offset + time_idx) % data_size]
            time_idx += 1

        # 勾配を求め、パラメータを更新
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1

    # エポックごとにパープレキシティの評価
    ppl = np.exp(total_loss / loss_count)
    print('| epoch %d | perplexity %.2f'
          % (epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0


corpus size: 1000, vocabulary size: 418
| epoch 1 | perplexity 405.89
| epoch 2 | perplexity 307.06
| epoch 3 | perplexity 229.56
| epoch 4 | perplexity 218.40
| epoch 5 | perplexity 206.74
| epoch 6 | perplexity 201.27
| epoch 7 | perplexity 198.80
| epoch 8 | perplexity 196.46
| epoch 9 | perplexity 191.61
| epoch 10 | perplexity 192.75
| epoch 11 | perplexity 188.81
| epoch 12 | perplexity 192.73
| epoch 13 | perplexity 190.44
| epoch 14 | perplexity 190.86
| epoch 15 | perplexity 190.34
| epoch 16 | perplexity 186.57
| epoch 17 | perplexity 183.87
| epoch 18 | perplexity 181.48
| epoch 19 | perplexity 182.93
| epoch 20 | perplexity 183.72
| epoch 21 | perplexity 181.61
| epoch 22 | perplexity 177.69
| epoch 23 | perplexity 175.13
| epoch 24 | perplexity 176.07
| epoch 25 | perplexity 172.63
| epoch 26 | perplexity 172.46
| epoch 27 | perplexity 167.59
| epoch 28 | perplexity 167.42
| epoch 29 | perplexity 163.07
| epoch 30 | perplexity 157.65
| epoch 31 | perplexity 158.38
| epoch 

In [ ]:
# グラフの描画
x = np.arange(len(ppl_list))
plt.plot(x, ppl_list, label='train')
plt.xlabel('epochs')
plt.ylabel('perplexity')
plt.show()

### 5.5.4 RNNLMのTrainerクラス

`RnnlmTrainer`クラスを作って、上で書いたRNNLMの学習を`RnnlmTrainer`クラスの内部に隠蔽する。
が、面白くないので略。

## 5.6 まとめ

本章のテーマはRNN。
RNNはデータを循環させることで、過去から現在、そして未来へとデータを継続して流す。
それによって、RNNレイヤ内部に「隠れ状態」を記憶する能力を得る。

RNNを利用したニューラルネットワークを構成することで、
理論的には、どれだけ長い時系列データであっても重要な情報をRNNの隠れ状態に記憶することが可能になる。
しかし、実際の問題ではうまく学習できないケースが多くある。
次章ではRNNの問題点を指摘し、RNNに代わる新しいレイヤ（LSTMレイヤやGRUレイヤ）を見ていく。